In [44]:
import time
import nltk;
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
from tabulate import tabulate
import pandas as pd
import nltk
import pyLDAvis.gensim
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
import gensim
import gensim.corpora as corpora
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import pyLDAvis
import json
import pandas as pd
import numpy as np
import re
import sys
import nltk
from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from rake_nltk import Rake
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import collections
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from stopwords_list import stop_words_list
#from wn_affect import wn_affect 
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from pyLDAvis import sklearn as sklearn_lda
from spacy.lang.en import English
import string
import re
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score
#!pip install en
from collections import defaultdict

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shwethailango/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shwethailango/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shwethailango/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
#hyperparameters & k-number of topics
k = 20
init = 'k-means++'
max_iter = 700
n_init = 10

In [47]:
from tabulate import tabulate


In [49]:
df=pd.read_csv('output.csv')
train_df=df.drop(columns=['Liked'], axis=1)
train_df

,Review
0,Wow... Loved this place.
1,Crust is not good.
2,Not tasty and the texture was just nasty.
3,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...
...,...
995,I think food should have flavor and texture an...
996,Appetite instantly gone.
997,Overall I was not impressed and would not go b...
998,"The whole experience was underwhelming, and I ..."


In [50]:
clean_df = train_df.dropna()

In [51]:
clean_df

,Review
0,Wow... Loved this place.
1,Crust is not good.
2,Not tasty and the texture was just nasty.
3,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...
...,...
995,I think food should have flavor and texture an...
996,Appetite instantly gone.
997,Overall I was not impressed and would not go b...
998,"The whole experience was underwhelming, and I ..."


In [52]:
clean_df.columns

Index(['Review'], dtype='object')

In [53]:
!pip install en

In [54]:
class TextPreProcessor():
    
    NLP = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    parser = English()
    noisy_pos_tags = ['PROP']
    min_token_length = 2


    def stopWords():
        spacy_stopwords = TextPreProcessor.NLP.Defaults.stop_words

        stop_words = stopwords.words('english')
        stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'need', 'able', 'bmc', 'com'])
        
        stop_words.extend(spacy_stopwords)
        return stop_words

    def removeStopWords(data):
        STOP_WORDS = TextPreProcessor.stopWords()
        cleanedData = []
        for sent in data:
            sent_cleaned = ""
            for word in sent.split():
                if word not in STOP_WORDS and word not in string.punctuation and len(word) > TextPreProcessor.min_token_length:
                    sent_cleaned += word
                    sent_cleaned += " "
            cleanedData.append(sent_cleaned)
        return cleanedData
                     
    def removePunctuation(data):
        return [sent.translate(sent.maketrans(string.punctuation, ' '*len(string.punctuation))) for sent in data]
    
    def removeHostName(data):
        cleanedData = []
        for sent in data:
            sent_cleaned = ""
            for word in sent.split():
                sent_cleaned += re.sub(r'^([A-Za-z0-9]\.|[A-Za-z0-9][A-Za-z0-9-]{0,61}[A-Za-z0-9]\.){1,3}[A-Za-z]{2,6}$', '', word)
                sent_cleaned += " "
            cleanedData.append(sent_cleaned)
        return cleanedData
    
    def removeIPAddress(data):
        cleanedData = []
        for sent in data:
            sent_cleaned = ""
            for word in sent.split():
                sent_cleaned += re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$', '', word)
                sent_cleaned += " "
            cleanedData.append(sent_cleaned)
        return cleanedData

    def removeEmails(data):
        return [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]

    def removeNewLineCharacters(data):
        return [re.sub(r'\s+', ' ', sent) for sent in data]

    def removeDigits(data):
        return [re.sub(r'\d', ' ', sent) for sent in data]
                
    def toLower(data):
        return [sent.lower() for sent in data]

    def spacy_tokenizer(sent):
        tokens = TextPreProcessor.parser(sent)
        tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_ for tok in tokens]
        tokens = [tok for tok in tokens if (tok not in STOP_WORDS and tok not in string.punctuation)]     
        return tokens


    def lemmatization(data, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
        stopWords = TextPreProcessor.stopWords()
        texts_out = []
        tokens_out = []
        for sent in data:
            #print(sent)
            doc = TextPreProcessor.NLP(sent) 
            lemmed_string =''
            tokens = []
            for word in doc:
                if word.pos_ in allowed_postags:
                    if word.lemma_ == '-PRON-' or word.lemma_ in stopWords: 
                        continue 
                    else: 
                        lemmed_string = lemmed_string + ' ' + word.lemma_
                        tokens.append(word.lemma_)
            #print(lemmed_string)
            texts_out.append(lemmed_string.strip())
            tokens_out.append(tokens)
        return tokens_out

    @staticmethod
    def cleanup(data):
        data = TextPreProcessor.toLower(data)
        data = TextPreProcessor.removeHostName(data)
        data = TextPreProcessor.removeIPAddress(data)
        data = TextPreProcessor.removeNewLineCharacters(data)
        data = TextPreProcessor.removePunctuation(data)
        data = TextPreProcessor.removeDigits(data)
        data = TextPreProcessor.removeEmails(data)
        data = TextPreProcessor.removeStopWords(data)
        data = TextPreProcessor.lemmatization(data)
        return data
    
    def __init__(self, df, columns, combine = False, dropCombinedColumns = False):
        self.df = df
        self.columns = columns
        self.combine = combine
        self.dropCombinedColumns = dropCombinedColumns
        self.process()
                  
    def process(self):  
        columnToProcess = self.columns                   
        if self.combine:
            self.df['combined'] = self.df[self.columns].apply(
                                        lambda x: ','.join(x.astype(str)), axis=1)
            if dropCombinedColumns:
                self.df = self.df.drop(self.columns, axis=1)
                columnToProcess = ["combined"]
            else:
                columnsToProcess.append("combined")
        for col in columnToProcess:
            print("Processing column : " + col)
            colName = col + "_clean"
            self.df[colName] = TextPreProcessor.cleanup(self.df[col])
        display(self.df.head())
        return self.df

In [55]:
import re
def process(data_df):
    preProcessor = TextPreProcessor(your_df, ['Review'])
    clean_df = preProcessor.df
    return clean_df

In [56]:
clean_df = process(clean_df)

Processing column : Review


,Review,Review_clean,Description_clean
0,Wow... Loved this place.,"[love, place]",love place
1,Crust is not good.,"[crust, good]",crust good
2,Not tasty and the texture was just nasty.,"[tasty, texture, nasty]",tasty texture nasty
3,Stopped by during the late May bank holiday of...,"[stopped, late, bank, holiday, rick, steve, re...",stopped late bank holiday rick steve recommend...
4,The selection on the menu was great and so wer...,"[selection, menu, great, price]",selection menu great price


In [57]:
msg_vec = clean_df['Review_clean']

In [58]:
msg_vec = msg_vec.apply(lambda x: str(x).replace('[','').replace(']','').replace(',','')) 

In [59]:
msg_vec = msg_vec.apply(lambda x: x.replace('\'',''))

In [60]:
clean_df['Description_clean'] = msg_vec

In [61]:
msg_vec.nunique()

959

In [62]:
clean_df

,Review,Review_clean,Description_clean
0,Wow... Loved this place.,"[love, place]",love place
1,Crust is not good.,"[crust, good]",crust good
2,Not tasty and the texture was just nasty.,"[tasty, texture, nasty]",tasty texture nasty
3,Stopped by during the late May bank holiday of...,"[stopped, late, bank, holiday, rick, steve, re...",stopped late bank holiday rick steve recommend...
4,The selection on the menu was great and so wer...,"[selection, menu, great, price]",selection menu great price
...,...,...,...
995,I think food should have flavor and texture an...,"[think, food, flavor, texture, lacking]",think food flavor texture lacking
996,Appetite instantly gone.,"[appetite, instantly]",appetite instantly
997,Overall I was not impressed and would not go b...,"[overall, impressed]",overall impressed
998,"The whole experience was underwhelming, and I ...","[experience, underwhelming, think, ninja, sush...",experience underwhelming think ninja sushi time


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [64]:
vectorizer = TfidfVectorizer(stop_words='english')

In [65]:
X = vectorizer.fit_transform(msg_vec)

In [66]:
X.shape[0]

1000

In [67]:
# run clustering algorihtm
import time
start_time = time.time()
true_k = k
k_means = KMeans(n_clusters=true_k, init=init, max_iter=max_iter, n_init=n_init)
kmeans_fit = k_means.fit(X)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.570131778717041 seconds ---


In [68]:
clusters = k_means.fit_predict(X)
centroids = k_means.cluster_centers_

In [69]:
import numpy as np
def k_mean_distance(data, centroid_matrix, i_centroid, cluster_labels):
    no_of_rec = data[cluster_labels == i_centroid].shape[0]
    distances = [np.linalg.norm(x - centroid_matrix) for x in data[cluster_labels == i_centroid]]
    wcss = sum(i * i for i in distances)
    if no_of_rec != 0:
        mean = wcss/no_of_rec
    else:
        mean = 0
  
    print('Cluster_id = {} & cluster_score = {} with No of records = {} & mean = {}'.format(i_centroid, wcss, no_of_rec, mean))
    return round(wcss, 2), no_of_rec, mean

In [70]:
def get_goodness_score(clusters, centroids, emb_matrix):
    print('Goodness score')
    score = []
    record_list = []
    mean_list = []
    for i, centroid_matrix in enumerate(centroids):
        cs, no_of_rec, mean = k_mean_distance(emb_matrix, centroid_matrix, i, clusters)
        score.append(cs)
        record_list.append(no_of_rec)
        mean_list.append(round(mean,2))
    return score, record_list, mean_list

In [71]:
score, record_list, mean_list = get_goodness_score(clusters, centroids, X)

Goodness score
Cluster_id = 0 & cluster_score = 130.89865157239936 with No of records = 133 & mean = 0.9842003877624012
Cluster_id = 1 & cluster_score = 43.5191046788152 with No of records = 54 & mean = 0.8059093459039851
Cluster_id = 2 & cluster_score = 13.972798203077916 with No of records = 21 & mean = 0.6653713430037103
Cluster_id = 3 & cluster_score = 24.13944810202232 with No of records = 30 & mean = 0.8046482700674107
Cluster_id = 4 & cluster_score = 15.526233846525987 with No of records = 21 & mean = 0.7393444688821899
Cluster_id = 5 & cluster_score = 38.649566485761774 with No of records = 42 & mean = 0.9202277734705184
Cluster_id = 6 & cluster_score = 15.065889123962386 with No of records = 18 & mean = 0.8369938402201326
Cluster_id = 7 & cluster_score = 61.45373944409728 with No of records = 74 & mean = 0.830455938433747
Cluster_id = 8 & cluster_score = 51.60897903674988 with No of records = 57 & mean = 0.9054206848552611
Cluster_id = 9 & cluster_score = 270.5473139471869 wit

In [72]:
from sklearn.metrics import silhouette_score
label = k_means.labels_

In [73]:
overall_score = silhouette_score(X, label, metric='euclidean')

In [75]:
# topic modeling
print("Top terms per cluster:")
val = []
order_centroids = k_means.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(k):
    print("Cluster %d:" % i)
    topic = ''
    for ind in order_centroids[i, :3]:
        topic = str(topic + ' ' + terms[ind]).strip()
    print("Label : {}".format(topic))

    val.append(topic.replace(' ', '-'))

Top terms per cluster:
Cluster 0:
Label : know awesome friendly
Cluster 1:
Label : great food service
Cluster 2:
Label : wait minute hour
Cluster 3:
Label : come time probably
Cluster 4:
Label : amazing place absolutely
Cluster 5:
Label : eat feel dish
Cluster 6:
Label : menu worth drive
Cluster 7:
Label : good food service
Cluster 8:
Label : food time think
Cluster 9:
Label : love delicious star
Cluster 10:
Label : tasty good extremely
Cluster 11:
Label : nice restaurant server
Cluster 12:
Label : terrible service food
Cluster 13:
Label : service slow fantastic
Cluster 14:
Label : salad bring ask
Cluster 15:
Label : place love recommend
Cluster 16:
Label : order arrive minute
Cluster 17:
Label : definitely pay worth
Cluster 18:
Label : bad service food
Cluster 19:
Label : night perfect family


In [76]:
topics=pd.DataFrame(val, columns=['Topics'])
topics

,Topics
0,know-awesome-friendly
1,great-food-service
2,wait-minute-hour
3,come-time-probably
4,amazing-place-absolutely
5,eat-feel-dish
6,menu-worth-drive
7,good-food-service
8,food-time-think
9,love-delicious-star


In [77]:
print(tabulate(topics, headers = 'keys', tablefmt = 'github'))


|    | Topics                   |
|----|--------------------------|
|  0 | know-awesome-friendly    |
|  1 | great-food-service       |
|  2 | wait-minute-hour         |
|  3 | come-time-probably       |
|  4 | amazing-place-absolutely |
|  5 | eat-feel-dish            |
|  6 | menu-worth-drive         |
|  7 | good-food-service        |
|  8 | food-time-think          |
|  9 | love-delicious-star      |
| 10 | tasty-good-extremely     |
| 11 | nice-restaurant-server   |
| 12 | terrible-service-food    |
| 13 | service-slow-fantastic   |
| 14 | salad-bring-ask          |
| 15 | place-love-recommend     |
| 16 | order-arrive-minute      |
| 17 | definitely-pay-worth     |
| 18 | bad-service-food         |
| 19 | night-perfect-family     |


In [78]:
cluster_results = pd.DataFrame()
for slot in clean_df.columns:
    cluster_results[slot] = clean_df[slot]
kmeans_fit.labels_.shape
cluster_results['cluster_id'] = kmeans_fit.labels_
cluster_results['cluster_labels'] = cluster_results['cluster_id'].apply(lambda x: val[x])
cluster_results['goodness_score'] = cluster_results['cluster_id'].apply(lambda x: mean_list[x])

cluster_results['cluster_id'] = cluster_results['cluster_id'].apply(lambda x: str(x))

In [79]:
cluster_results.sort_values(by='cluster_id', inplace=True)

In [80]:
cluster_results

,Review,Review_clean,Description_clean,cluster_id,cluster_labels,goodness_score
169,Bacon is hella salty.,"[bacon, hella, salty]",bacon hella salty,0,know-awesome-friendly,0.98
542,The yellowtail carpaccio was melt in your mout...,"[yellowtail, carpaccio, melt, mouth, fresh]",yellowtail carpaccio melt mouth fresh,0,know-awesome-friendly,0.98
548,The desserts were a bit strange.,"[dessert, bit, strange]",dessert bit strange,0,know-awesome-friendly,0.98
821,Do not waste your money here!,"[waste, money]",waste money,0,know-awesome-friendly,0.98
555,I know this is not like the other restaurants ...,"[know, restaurant]",know restaurant,0,know-awesome-friendly,0.98
...,...,...,...,...,...,...
636,Tried to go here for lunch and it was a madhouse.,"[try, lunch, madhouse]",try lunch madhouse,9,love-delicious-star,0.93
635,We were promptly greeted and seated.,"[promptly, greet, seated]",promptly greet seated,9,love-delicious-star,0.93
632,"Same evening, him and I are both drastically s...","[evening, drastically, sick]",evening drastically sick,9,love-delicious-star,0.93
627,Any grandmother can make a roasted chicken bet...,"[grandmother, roast, chicken]",grandmother roast chicken,9,love-delicious-star,0.93


------